In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

import numpy as np
import scipy.linalg as la
from scipy.spatial import cKDTree
import scipy.sparse as sp

from poly_basis import *
from spherepts import *
from rbf import *

# Tangent Plane Method

In [9]:
#def poly_basis_2D()

def dist_outer_W(nodes1, nodes2, W):
    d = len(nodes1[0]) # the dimension of each vector
    n1 = len(nodes1)
    n2 = len(nodes2)
    # create a row vector of d-dimensional vectors
    row = nodes1.reshape((1,n1,d)) 
    # create a column vector of d-dimensional vectors
    col = nodes2.reshape((n2,1,d)) 
    diff = row-col
    ret = diff @ W * diff
    #ret = (row-col)**2
    ret = np.sum(ret,2) #sum each d-dimensional vector
    return np.sqrt(ret)

# calculate the weights using the tangent plane method.
def weights_tan(nodes, rbf, Lrbf, eps=None, normals=None, stencil_size=10):    
    n = len(nodes)
    tree = cKDTree(np.array(nodes))

    e1, e2, e3 = np.eye(3)
    E = np.eye(3)
    E[2,2] = 0
    
    # Calculate Weights
    weights = np.zeros((n, stencil_size))
    row_index = [r for r in range(n) for c in range(stencil_size)]
    col_index = np.zeros((n, stencil_size))
    for i, node in enumerate(nodes):
        stencil = tree.query(node, stencil_size)[1]
        col_index[i] = stencil
        
        nn = np.array([nodes[i] for i in stencil])
        
        W = np.zeros((3, 3))
        R = np.zeros((3, 3))
        
        t1 = e2 - np.dot(node, e2)*node
        t1 /= la.norm(t1)
        t2 = e3 - np.dot(node, e3)*node - np.dot(t1, e3)*t1
        t2 /= la.norm(t2)
        R[:,0] = t1
        R[:,1] = t2
        R[:,2] = node
        W = R @ E @ R.T
        
        if eps is None:
            eps = optimize_eps(rbf, dist_outer_W(nn, nn, W))
        
        A = rbf(dist_outer_W(nn, nn, W), eps)
        rhs = Lrbf(dist_outer_W(nn,nn[0].reshape((1,3)), W), eps)
        weights[i] = la.solve(A, rhs.flatten())
    C = sp.csc_matrix((weights.ravel(), (row_index, col_index.ravel())),shape=(n,n))
    return C

# Projection Method

In [10]:
def grad_rbf_outer(zeta, eps, nodes, centers):
    n_len = len(nodes)
    c_len = len(centers)
    r = dist_outer(nodes, centers)[:,:,np.newaxis]
    xs = (np.array(nodes).reshape((1,n_len,3)) - np.array(centers).reshape((c_len,1,3)))
    return zeta(r, eps) * xs

def weights_proj(nodes, rbf, zeta, eps=None, normals=None, stencil_size=10, basis_type='s', basis_deg=-1):
    n = len(nodes)
    tree = cKDTree(np.array(nodes))
    projectors = [np.eye(3) - np.outer(node, node) for node in nodes]
    
    weights = np.zeros((n, stencil_size))
    row_index = [r for r in range(n) for c in range(stencil_size)]
    col_index = np.zeros((n, stencil_size))
    for i, node in enumerate(nodes):
        stencil = tree.query(node, stencil_size)[1]
        col_index[i] = stencil
        nn = np.array([nodes[i] for i in stencil])
        
        nn_proj = np.array([projectors[i] for i in stencil])
        center_x, center_y, center_z = nn[0]

        if basis_type == 'p':
            P, rhs_x, rhs_y, rhs_z = get_poly_basis(basis_deg, nn, nn_proj)
        elif basis_type == 's':
            P, rhs_x, rhs_y, rhs_z = gen_sphere_harm_basis(basis_deg, nn, nn_proj)

        dist_mat = dist_outer(nn,nn)
        if eps is None:
            eps = optimize_eps(rbf, dist_mat)

        A = rbf(dist_mat, eps)

        num_basis = P.shape[1]
        AP = np.block([[A,P],[P.T, np.zeros((num_basis,num_basis))]])

        rhsAs = np.matmul(nn_proj, 
                          grad_rbf_outer(zeta, eps, nn, nn).reshape(
                            (stencil_size,stencil_size,3,1))).reshape(
                            (stencil_size,stencil_size,3))

        rhsA = rhsAs[:,:,0] # only the x coordinates
        rhs = np.block([[rhsA],
                        [rhs_x]])
        weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
        weights[i] = (weights_grad@weights_grad)[0]

        rhsA = rhsAs[:,:,1] # only the y coordinates
        rhs = np.block([[rhsA],
                        [rhs_y]])
        weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
        weights[i] += (weights_grad@weights_grad)[0]

        rhsA = rhsAs[:,:,2] # only the z coordinates
        rhs = np.block([[rhsA],
                        [rhs_z]])
        weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
        weights[i] += (weights_grad@weights_grad)[0]

    C = sp.csc_matrix((weights.ravel(), (row_index, col_index.ravel())),shape=(n,n))
    return C

In [11]:
# Choose solution and forcing fuction
solution_index = 10
sol_deg = sphere_harm_degs[solution_index]
exact = lambda x: sphere_harm[solution_index](*x)*-sol_deg*(sol_deg+1)
forcing = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sol_deg)

Harmonic degree: 3


In [12]:
n = 2000
k = 100
eps = .25
rbf_obj=rbf_dict['multiquadric']


rbf = rbf_obj['rbf']
zeta  = rbf_obj['zeta']
d2rbf = rbf_obj['d2rbf']
Lrbf = lambda r,eps: 1*zeta(r,eps) + d2rbf(r,eps)
def Lpoly(P):
    ret = np.zeros((P.shape[1],1))
    if len(ret) >= 3:
        ret[3,0] = 2
    if len(ret) >= 5:
        ret[5,0] = 2
    return ret

# generate nodes
nodes = gen_spiral_nodes(n)
true = np.array([exact(node) for node in nodes])
fs = np.array([forcing(node) for node in nodes])

In [13]:
# Tangent Plane Method
C = weights_tan(nodes, rbf, Lrbf, normals=nodes, eps=None, stencil_size=k)
print(la.norm(true - C@fs)/la.norm(true))

7.487853926857502e-05


In [14]:
# Projection Method
C = weights_proj(nodes, rbf, zeta, normals=nodes, eps=None, stencil_size=k, basis_deg=-1)
print(la.norm(true - C@fs)/la.norm(true))

0.0002886258011746915


In [23]:
stencil_size = k
e1, e2, e3 = np.eye(3)
E = np.eye(3)
E[2,2] = 0

i = 1
node = nodes[i]
tree = cKDTree(np.array(nodes))
stencil = tree.query(node, stencil_size)[1]

nn = np.array([nodes[i] for i in stencil])

W = np.zeros((3, 3))
R = np.zeros((3, 3))

t1 = e2 - np.dot(node, e2)*node
t1 /= la.norm(t1)
t2 = e3 - np.dot(node, e3)*node - np.dot(t1, e3)*t1
t2 /= la.norm(t2)
R[:,0] = t1
R[:,1] = t2
R[:,2] = node

In [24]:
R

array([[ 1.18985754e-03,  9.98793139e-01, -4.91003951e-02],
       [ 9.99706506e-01, -7.06395271e-17,  2.42260439e-02],
       [-2.41968064e-02,  4.91148100e-02,  9.98500000e-01]])

In [27]:
nodes @ R

array([[-5.36321943e-02,  6.05342922e-02,  9.96724228e-01],
       [ 0.00000000e+00,  3.88578059e-16,  1.00000000e+00],
       [-4.30983530e-03,  1.16760167e-01,  9.93150789e-01],
       ...,
       [ 3.43460662e-02,  2.08602805e-02, -9.99192272e-01],
       [-1.82576542e-02, -8.36531901e-02, -9.96327658e-01],
       [ 5.57420507e-02, -5.08673211e-02, -9.97148604e-01]])

In [28]:
nodes @ R[:,:2]

array([[-5.36321943e-02,  6.05342922e-02],
       [ 0.00000000e+00,  3.88578059e-16],
       [-4.30983530e-03,  1.16760167e-01],
       ...,
       [ 3.43460662e-02,  2.08602805e-02],
       [-1.82576542e-02, -8.36531901e-02],
       [ 5.57420507e-02, -5.08673211e-02]])

In [36]:
P = np.zeros((10,4))

In [38]:
P.shape[1]

4

In [39]:
np.__path__

['/usr/local/lib/python3.6/dist-packages/numpy']